In [2]:
from pathlib import Path as P
import fiftyone as fo


## Load VOD dataset annotation 

In [3]:
vod_datapath = P('../../data/vod/')
tag = 'radar'
vod_radarpath = vod_datapath / tag
sorted(vod_radarpath.glob('*'))

[PosixPath('../../data/vod/radar/.DS_Store'),
 PosixPath('../../data/vod/radar/ImageSets'),
 PosixPath('../../data/vod/radar/README.md'),
 PosixPath('../../data/vod/radar/clips'),
 PosixPath('../../data/vod/radar/clips_info.yaml'),
 PosixPath('../../data/vod/radar/convert_to_radar_format.py'),
 PosixPath('../../data/vod/radar/gt_database'),
 PosixPath('../../data/vod/radar/kitti_dbinfos_train.pkl'),
 PosixPath('../../data/vod/radar/kitti_infos_test.pkl'),
 PosixPath('../../data/vod/radar/kitti_infos_train.pkl'),
 PosixPath('../../data/vod/radar/kitti_infos_trainval.pkl'),
 PosixPath('../../data/vod/radar/kitti_infos_val.pkl'),
 PosixPath('../../data/vod/radar/testing'),
 PosixPath('../../data/vod/radar/training')]

In [8]:
# load pickle 
import pickle
with open('../../data/vod/radar/kitti_infos_train.pkl', 'rb') as f:
    test_pickle = pickle.load(f)

In [9]:
dir(test_pickle)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [13]:
info = test_pickle[0]

In [7]:
!pwd

/mnt/12T/DJ/RC_fusion/CenterFusion/src/tools


In [11]:
len(test_pickle)

5139

In [12]:
img_dir = '../../data/vod/radar/training/image_2/'

In [14]:
height, width = info['image']['image_shape']


In [15]:
height


1216

In [16]:
width

1936

In [19]:
info['image']['image_idx']

'00544'

In [20]:
info

{'point_cloud': {'num_features': 7, 'lidar_idx': '00544'},
 'image': {'image_idx': '00544',
  'image_shape': array([1216, 1936], dtype=int32)},
 'calib': {'P2': array([[1.49546863e+03, 0.00000000e+00, 9.61272461e+02, 0.00000000e+00],
         [0.00000000e+00, 1.49546863e+03, 6.24895935e+02, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'R0_rect': array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]], dtype=float32),
  'Tr_velo_to_cam': array([[-0.013857  , -0.9997468 ,  0.01772762,  0.05283124],
         [ 0.10934269, -0.01913807, -0.99381983,  0.98100483],
         [ 0.99390751, -0.01183297,  0.1095802 ,  1.44445002],
         [ 0.        ,  0.        ,  0.        ,  1.        ]])},
 'annos': {'name': array(['bicycle', 'bicycle', 'bicycle_rack', 'bicycle', 'Cyclist',
         'Cyclist', 'Cyclist', 'moped_scoote

In [21]:
annos = info['annos']

In [22]:
annos

{'name': array(['bicycle', 'bicycle', 'bicycle_rack', 'bicycle', 'Cyclist',
        'Cyclist', 'Cyclist', 'moped_scooter', 'Pedestrian', 'Pedestrian',
        'rider', 'rider', 'rider', 'bicycle', 'moped_scooter',
        'Pedestrian'], dtype='<U13'),
 'truncated': array([0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.]),
 'occluded': array([0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 2., 1., 1., 2., 0.]),
 'alpha': array([-1.69352575,  1.99899904,  0.33835665, -1.88022201, -2.00432832,
        -0.20291482,  3.03977651, -0.16560054, -2.92064428,  0.23600459,
        -2.00685064,  2.95821659, -0.20052092,  2.07547287,  0.36338399,
        -2.95515474]),
 'bbox': array([[1204.9718 ,  766.78656, 1318.0629 ,  931.94824],
        [   0.     ,  669.8923 ,  278.69495,  920.9737 ],
        [ 199.3622 ,  762.29047,  392.51443,  871.6183 ],
        [1425.1237 ,  759.5363 , 1712.0964 , 1117.4009 ],
        [ 829.68066,  704.99744, 1048.8776 , 1013.1166 ],
        [ 905.1772 ,  748.

In [25]:
annos = info['annos']
bbox = annos['bbox']
alpha = annos['alpha']
dimensions = annos['dimensions']
yaw = annos['rotation_y']
location = annos['location']
name = annos['name']

In [27]:
used_classes = ['Car', 'Cyclist', 'Pedestrian']

In [30]:
import numpy as np
def keep_arrays_by_name(gt_names, used_classes):
    inds = [i for i, x in enumerate(gt_names) if x in used_classes]
    inds = np.array(inds, dtype=np.int64)
    return inds

In [38]:
selected_name = annos['name']
selected = keep_arrays_by_name(selected_name, used_classes)
selected_annos = {}
for k in annos.keys():
    selected_annos[k] = annos[k][selected]

bbox = selected_annos['bbox']
alpha = selected_annos['alpha']
dimensions = selected_annos['dimensions']
yaw = selected_annos['rotation_y']
location = selected_annos['location']
name = selected_annos['name']

In [32]:
selected

array([ 4,  5,  6,  8,  9, 15])

In [42]:
obj_id = np.array([used_classes.index(n) + 1 for n in name], dtype=np.int32)

In [40]:
gt_classes

array([2, 2, 2, 3, 3, 3], dtype=int32)

In [41]:
name

array(['Cyclist', 'Cyclist', 'Cyclist', 'Pedestrian', 'Pedestrian',
       'Pedestrian'], dtype='<U13')

In [43]:
obj_id.shape

(6,)

In [44]:
bbox.shape

(6, 4)

In [47]:
def get_coco_bbox(bboxes):
    bbox_num = bboxes.shape[0]
    coco_bboxes = np.zeros_like(bboxes)
    for i in range(bbox_num):
        bbox = bboxes[i,:]
        coco_bboxes[i,:] = np.array([bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]])
    return coco_bboxes

In [48]:
get_coco_bbox(bbox)

array([[829.68066 , 704.99744 , 219.1969  , 308.11914 ],
       [905.1772  , 748.95575 , 175.50323 , 161.84235 ],
       [382.7782  , 711.4978  , 166.60754 , 144.18585 ],
       [574.0226  , 725.1997  ,  69.59619 , 128.61609 ],
       [410.69986 , 677.243   , 105.18991 , 182.99591 ],
       [632.68866 , 742.02765 ,  61.234802, 114.66333 ]], dtype=float32)

In [49]:
yaw

array([-2.02526056, -0.18243772, -3.56297328, -3.1517889 , -0.08453429,
       -3.1515679 ])

In [50]:
anns_num = len(obj_id)

In [ ]:
anns_list = []
for i in range(anns_num):
    ann_dict = {
        'id':0,
        'image_id':,
        'category_id':,
        'dim':,
        'location':,
        'depth':,
        'occluded':,
        'truncated':,
        'rotation_y':,
        'amodel_center':,
        'iscrowd':,
        'track_id':,
        'attributes':,
        'velocity':,
        'velocity_cam':
        'bbox':,
        'area':,
        'alpha':
    }
    anns_list += [ann_dict]

In [51]:
def get_amodel(location, kitti_yaw, dims):

    # Extract the location and yaw information from the KITTI bounding box
    x, y, z = location
    yaw = kitti_yaw

    # Convert the yaw to the rotation matrix
    cos_yaw = np.cos(yaw)
    sin_yaw = np.sin(yaw)
    rotation_matrix = np.array([[cos_yaw, 0, -sin_yaw], [0, 1, 0], [sin_yaw, 0, cos_yaw]])

    # Convert the location to the amodel_center format
    amodel_center = np.dot(rotation_matrix.T, np.array([x, y, z])) - np.dot(rotation_matrix.T, dims) / 2
    return amodel_center

test_dim = dimensions[0]
test_loc = location[0]
test_yaw = yaw[0]
get_amodel(test_loc, test_yaw, test_dim)

array([-8.3507115 ,  1.48758718, -5.56305806])

In [ ]:
info['calib']